In [1]:
#!pip install surprise
#!pip install import-ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import pickle
#sns.set_style("darkgrid")
from surprise import accuracy
from collections import defaultdict
from surprise import KNNBasic

from collections import defaultdict
from operator import itemgetter
import heapq
import os
import csv

In [2]:
use_pickle_file = True
max_n = 2500000 #how many rows we want from data_ratings and data_rating_plus_movie_title


## Helper functions

In [3]:
def all_movie_id_rows(movie_id):
    return data_rating[data_rating['movie_id'] == movie_id]

def all_custumers_id_rows(customer_id):
    return data_rating[data_rating['customer_id'] == customer_id]

def all_customers_id_plus_movie_title_rows(customer_id):
    return data_rating_plus_movie_title[data_rating_plus_movie_title['customer_id'] == customer_id]

def all_movies_get_average_rating():
    movie_ratings_stats = data_rating.groupby('movie_id').agg({'rating': ['sum', 'count']}).reset_index()
    movie_ratings_stats['avg_rating'] =  movie_ratings_stats['rating']['sum'] / movie_ratings_stats['rating']['count']
    return movie_ratings_stats

def all_customers_get_average_rating():
    customers_ratings_stats = data_rating.groupby('customer_id').agg({'rating': ['sum', 'count']}).reset_index()
    customers_ratings_stats['avg_rating'] =  customers_ratings_stats['rating']['sum'] / customers_ratings_stats['rating']['count']
    return customers_ratings_stats

def all_movies_get_rated_count():
    return data_rating.groupby('movie_id').agg({'movie_id': 'count'}).reset_index()

def all_customers_get_movie_rated_count():
    return data_rating.groupby('customer_id').agg({'movie_id': 'count'}).reset_index()

def get_customers_avg_rating_less_than(max_rating):
    print(all_customers_average_ratings[all_customers_average_ratings['avg_rating'] < max_rating])

def get_movies_avg_rating_less_than(max_rating):
    print(all_movies_average_rating[all_movies_average_rating['avg_rating'] < max_rating])

def get_customers_avg_rating_higher_than(min_rating):
    print(all_customers_average_ratings[all_customers_average_ratings['avg_rating'] > min_rating])

def get_movies_avg_rating_higher_than(min_rating):
    print(all_movies_average_rating[all_movies_average_rating['avg_rating'] > min_rating])


def get_movie_avg_rating(movie_id):
    output = all_movies_average_rating[all_movies_average_rating['movie_id'] == movie_id]
    #print(output)
    return output['avg_rating']

def get_users_avg_rating(customer_id):
    output = all_customers_average_ratings[all_customers_average_ratings['customer_id'] == customer_id]
    #print(output)
    return output['avg_rating']

def get_movies_customer_rated_higher_than(customer_id, min_rating=4):
    temp = data_rating_plus_movie_title.copy()
    return temp[(temp['customer_id'] == customer_id) & (temp['rating'] >= min_rating)].set_index('movie_id')

def get_movies_customer_rated_lower_than(customer_id, max_rating=4):
    temp = data_rating_plus_movie_title.copy()
    return temp[(temp['customer_id'] == customer_id) & (temp['rating'] < max_rating)].set_index('movie_id')

def display_movies_customer_rated_higher_than(customer_id, min_rating=0):
    df_customer_liked = get_movies_customer_rated_higher_than(customer_id=customer_id, min_rating=min_rating)
    print(df_customer_liked[['movie_title', 'rating']])
    print('average rating', get_users_avg_rating(customer_id))

def display_movies_customer_rated_lower_than(customer_id, max_rating=0):
    df_customer_disliked = get_movies_customer_rated_lower_than(customer_id=customer_id, max_rating=max_rating)
    print(df_customer_disliked[['movie_title', 'rating']])
    print('average rating', get_users_avg_rating(customer_id))

def save_to_pickle(name, df):
    path_name = "pickle/"+name+".pickle"
    pickle_file = open(path_name,"wb")
    pickle.dump(df, pickle_file)
    pickle_file.close()

def load_pickle(name):
    path_name = "pickle/"+name+".pickle"
    return_input = open(path_name, "rb")
    return pickle.load(return_input)


## Get the data

In [4]:
def create_dataframe(use_pickle=True):
    if(use_pickle == True):
        in_pickle = open("pickle/movies_customers_ratings.pickle", "rb")
        data = pd.DataFrame(pickle.load(in_pickle), columns=['movie_id', 'customer_id', 'rating', 'index']).drop(['index'], axis=1)
    else:
        index = 1
        last_movie_id = "1"
        new_data = []
        ## import all combined_data files to one large pandas dataframe
        ##> returns index, customer_id (which is both movie and customer), rating (NaN = customer_id is a movie, Not NaN = customers rating)
        df_all = pd.read_csv('./data/combined_data_1.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1])
        #df_all = df_all.append(pd.read_csv('./data/combined_data_2.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1]))
        #df_all = df_all.append(pd.read_csv('./data/combined_data_3.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1]))
        #df_all = df_all.append(pd.read_csv('./data/combined_data_4.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1]))
        df_all.index = np.arange(0,len(df_all))
        df_all['rating'] = df_all['rating'].astype(float)
        
        for customer_id in df_all["customer_id"]:
            # if we find : that means this is a movie_id and not customer_id
            if(customer_id.find(":") > 0):
                movie_id = customer_id.replace(":", "")
                last_movie_id = movie_id
            else:
                # we have this row index so use it to get rating
                rating = df_all["rating"][index-1]
                new_data.append([last_movie_id, customer_id, rating, index])
            index += 1
        #output to pickle file
        movies_customers_ratings = open("pickle/movies_customers_ratings.pickle","wb")
        pickle.dump(new_data, movies_customers_ratings)
        movies_customers_ratings.close()
        data = pd.DataFrame(new_data, columns=['movie_id', 'customer_id', 'rating', 'index']).drop(['index'], axis=1)

    # change columns to numerical
    data['movie_id'] = data['movie_id'].astype(int)
    data['customer_id'] = data['customer_id'].astype(int)
    data["rating"] = data["rating"].astype(float)
    return data

def get_dataframes(use_pickle=True):
    # dataframe containing all informations about the movies
    #> returns movie_id, movie_year, movie_title
    data_movies = pd.read_csv('./data/movie_titles.csv', header = None, names = ['movie_id', 'movie_year', 'movie_title'], usecols = [0,1,2], encoding="latin1")
    #data_movies_import.set_index('movie_id', inplace = True)

    ## ------------------------------------------------------------------------------------- ##

    # dataframe containing all informations about the movie ratings by customer
    #> returns index, movie_id, customer_id, rating
    data_rating = create_dataframe(use_pickle=use_pickle)

    # ## ------------------------------------------------------------------------------------- ##

    # ##combine customer ratings to movie titles
    # ##> returns index, movie_id, customer_id, rating, movie_year, movie_title
    data_rating_plus_movie_title = data_rating.merge(data_movies, on="movie_id", how="inner")

    ## ------------------------------------------------------------------------------------- ##

    data_movies_categorize = pd.read_csv('./data/movies.csv', header = None, names = ['movie_id', 'movie_title', 'genres'], usecols = [0,1,2], encoding="latin1")[1:] #dataset is off by one
    #data_movies_categorize.set_index('movie_id', inplace = True)
    data_movies_categorize_split = data_movies_categorize['movie_title'].str.split('(', n = 1, expand=True) # split movie_title to movie_title and movie_year
    data_movies_categorize_split[1] = data_movies_categorize_split[1].str.replace(r')', '') #removing ) at the end of movie_year
    data_movies_categorize["movie_year"] = data_movies_categorize_split[1]
    data_movies_categorize["movie_title"] = data_movies_categorize_split[0]
    #data_movies_categorize["movie_year"] = data_movies_categorize["movie_year"].astype(float)
    #data_movies_categorize
    data_movies_categorize_cleaned = data_movies_categorize[pd.to_numeric(data_movies_categorize['movie_year'], errors='coerce').notnull()]
    data_movies_categorize_cleaned["movie_year"] = data_movies_categorize_cleaned["movie_year"].astype(float)
    
    return data_movies, data_rating, data_rating_plus_movie_title, data_movies_categorize_cleaned


def get_users_loved_hated_movies(customer_id, minmax_rating):
    users_ratings_higher_than_four = get_movies_customer_rated_higher_than(customer_id=customer_id_use, min_rating=minmax_rating)
    users_ratings_lower_than_four = get_movies_customer_rated_lower_than(customer_id=customer_id_use, max_rating=minmax_rating)
    print("User", customer_id_use ,"loved these movies")
    for rating in users_ratings_higher_than_four['movie_title']:
        print(rating)
    print('')
    print("and disliked these movies")
    for rating in users_ratings_lower_than_four['movie_title']:
        print(rating)

in these txt files the movie_id's have been placed amongst the customer_id's, which means that we have to split that column into two, we do this by taking all ratings that are equal to NaN 
( meaning that we are now refering to a movie and not a customer ) and from there we split it into two columns.

In [5]:
data_movies, data_rating, data_rating_plus_movie_title, data_movies_categorized = get_dataframes(use_pickle=use_pickle_file)

<ipython-input-4-53fccbde2917>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movies_categorize_cleaned["movie_year"] = data_movies_categorize_cleaned["movie_year"].astype(float)


In [6]:
data_rating = data_rating[:max_n]
data_rating_plus_movie_title = data_rating_plus_movie_title[:max_n]

#### try again later

In [7]:
# # temporarly while join with genre not working correctly
# data_movies = data_movies_import.copy()
# data_movies.set_index('movie_id', inplace = True)
# print(data_movies)

In [8]:
# x_temp = data_movies_import
# x2_temp= data_movies_categorize
# #x_temp.set_index("movie_id", inplace=True)
# #x2_temp.set_index("movie_id", inplace=True)

In [9]:
# temp_merge = pd.merge(x_temp, x2_temp, on=['movie_title', 'movie_year'], how='outer')
# temp_merge = temp_merge[["movie_year", "movie_title"]].drop_duplicates(subset=None, keep="last", inplace=False)

In [10]:
# df_merged = pd.merge(x_temp, x2_temp, on=["movie_title", "movie_year"], how="left")

In [11]:
#df_merged["genres"].isnull()

In [12]:
# #data_movies = pd.merge(x, x2, on=['movie_year', 'movie_title'], how='outer')
# data_movies = pd.merge(x, x2, on=["movie_title", "movie_year"], how='outer')
# data_movies#.set_index('movie_id', inplace = True)


In [13]:
# data_movies.groupby("movie_title").count()

#### end of try again later

## Working with the data

### Details about all customers/movies

In [14]:
# get the average movie rating for all customers
# used to determine if this user typically gives bad or good reviews
# and then we can see if he really hates or loves a movie
all_customers_average_ratings = all_customers_get_average_rating()
print(all_customers_average_ratings)

       customer_id rating       avg_rating
                      sum count           
0                6   45.0    13   3.461538
1                7   83.0    18   4.611111
2               10   16.0     5   3.200000
3               25    3.0     1   3.000000
4               33    4.0     1   4.000000
...            ...    ...   ...        ...
360194     2649401   11.0     3   3.666667
360195     2649404    7.0     2   3.500000
360196     2649409   15.0     4   3.750000
360197     2649426   47.0    11   4.272727
360198     2649429   23.0     6   3.833333

[360199 rows x 4 columns]


In [15]:
all_movies_average_rating = all_movies_get_average_rating()
print(all_movies_average_rating)

    movie_id  rating       avg_rating
                 sum count           
0          1  2051.0   547   3.749543
1          2   516.0   145   3.558621
2          3  7326.0  2012   3.641153
3          4   389.0   142   2.739437
4          5  4468.0  1140   3.919298
..       ...     ...   ...        ...
463      464   226.0    82   2.756098
464      465   292.0   100   2.920000
465      466  1357.0   362   3.748619
466      467  3759.0  1055   3.563033
467      468  4062.0  1158   3.507772

[468 rows x 4 columns]


In [55]:
get_customers_avg_rating_less_than(max_rating=4)

       customer_id rating       avg_rating
                      sum count           
0                6   45.0    13   3.461538
2               10   16.0     5   3.200000
3               25    3.0     1   3.000000
5               42   14.0     4   3.500000
6               59   25.0     7   3.571429
...            ...    ...   ...        ...
360193     2649388   22.0     7   3.142857
360194     2649401   11.0     3   3.666667
360195     2649404    7.0     2   3.500000
360196     2649409   15.0     4   3.750000
360198     2649429   23.0     6   3.833333

[207379 rows x 4 columns]


In [17]:
get_customers_avg_rating_less_than(max_rating=4)

       customer_id rating       avg_rating
                      sum count           
0                6   45.0    13   3.461538
2               10   16.0     5   3.200000
3               25    3.0     1   3.000000
5               42   14.0     4   3.500000
6               59   25.0     7   3.571429
...            ...    ...   ...        ...
360193     2649388   22.0     7   3.142857
360194     2649401   11.0     3   3.666667
360195     2649404    7.0     2   3.500000
360196     2649409   15.0     4   3.750000
360198     2649429   23.0     6   3.833333

[207379 rows x 4 columns]


In [18]:
get_customers_avg_rating_higher_than(min_rating=4)

       customer_id rating       avg_rating
                      sum count           
1                7   83.0    18   4.611111
13             126    5.0     1   5.000000
15             134   83.0    18   4.611111
19             168    9.0     2   4.500000
27             199   76.0    18   4.222222
...            ...    ...   ...        ...
360181     2649311   29.0     7   4.142857
360184     2649328   21.0     5   4.200000
360186     2649335   41.0    10   4.100000
360190     2649375   30.0     7   4.285714
360197     2649426   47.0    11   4.272727

[92187 rows x 4 columns]


In [19]:
get_movies_avg_rating_less_than(max_rating=5)

    movie_id  rating       avg_rating
                 sum count           
0          1  2051.0   547   3.749543
1          2   516.0   145   3.558621
2          3  7326.0  2012   3.641153
3          4   389.0   142   2.739437
4          5  4468.0  1140   3.919298
..       ...     ...   ...        ...
463      464   226.0    82   2.756098
464      465   292.0   100   2.920000
465      466  1357.0   362   3.748619
466      467  3759.0  1055   3.563033
467      468  4062.0  1158   3.507772

[468 rows x 4 columns]


In [20]:
get_movies_avg_rating_less_than(max_rating=4)

    movie_id  rating       avg_rating
                 sum count           
0          1  2051.0   547   3.749543
1          2   516.0   145   3.558621
2          3  7326.0  2012   3.641153
3          4   389.0   142   2.739437
4          5  4468.0  1140   3.919298
..       ...     ...   ...        ...
463      464   226.0    82   2.756098
464      465   292.0   100   2.920000
465      466  1357.0   362   3.748619
466      467  3759.0  1055   3.563033
467      468  4062.0  1158   3.507772

[443 rows x 4 columns]


In [21]:
count_movies_rated_by_customer = all_customers_get_movie_rated_count()
count_movies_rated_by_customer

,customer_id,movie_id
0,6,13
1,7,18
2,10,5
3,25,1
4,33,1
...,...,...
360194,2649401,3
360195,2649404,2
360196,2649409,4
360197,2649426,11


In [22]:
def generate_random_user_ids(from_n, to_n):
    ids = np.unique(data_rating['customer_id'])[from_n:to_n]
    print("random user id used while in development\n")
    return ids

In [23]:
generate_random_user_ids(1000, 1500)

random user id used while in development



array([ 7410,  7411,  7459,  7461,  7462,  7497,  7521,  7524,  7538,
        7544,  7548,  7550,  7554,  7576,  7577,  7579,  7580,  7581,
        7594,  7599,  7601,  7602,  7613,  7617,  7620,  7623,  7628,
        7634,  7652,  7659,  7661,  7677,  7683,  7703,  7708,  7714,
        7715,  7730,  7745,  7748,  7751,  7766,  7770,  7775,  7786,
        7797,  7798,  7799,  7801,  7805,  7806,  7807,  7815,  7837,
        7839,  7846,  7851,  7867,  7869,  7877,  7882,  7889,  7899,
        7903,  7904,  7908,  7910,  7911,  7921,  7927,  7940,  7942,
        7949,  7953,  7956,  7963,  7964,  7968,  7997,  8002,  8004,
        8012,  8023,  8026,  8032,  8043,  8053,  8055,  8056,  8061,
        8074,  8079,  8081,  8092,  8095,  8102,  8117,  8118,  8121,
        8130,  8135,  8138,  8141,  8143,  8158,  8160,  8167,  8170,
        8174,  8175,  8180,  8183,  8185,  8191,  8198,  8200,  8205,
        8207,  8209,  8210,  8212,  8219,  8221,  8226,  8230,  8270,
        8274,  8275,

### Details about a specific customer

In [24]:
# ids I like -> 28812, 56514, 56520, 1488844
customer_id_use = 84100

In [25]:
#all_custumers_id_rows(customer_id=customer_id_use)
## same as line above but including movie_year and movie_title
all_customers_id_plus_movie_title_rows(customer_id=customer_id_use)

,movie_id,customer_id,rating,movie_year,movie_title
141847,30,84100,5.0,2003.0,Something's Gotta Give
757653,191,84100,4.0,2003.0,X2: X-Men United
1408832,299,84100,4.0,2001.0,Bridget Jones's Diary
1936216,357,84100,5.0,2003.0,House of Sand and Fog
2417368,457,84100,4.0,2004.0,Kill Bill: Vol. 2


In [26]:
get_users_avg_rating(customer_id=customer_id_use)

11376    4.4
Name: avg_rating, dtype: float64

In [27]:
display_movies_customer_rated_higher_than(customer_id=customer_id_use, min_rating=4)

                     movie_title  rating
movie_id                                
30        Something's Gotta Give     5.0
191             X2: X-Men United     4.0
299        Bridget Jones's Diary     4.0
357        House of Sand and Fog     5.0
457            Kill Bill: Vol. 2     4.0
average rating 11376    4.4
Name: avg_rating, dtype: float64


In [28]:
display_movies_customer_rated_lower_than(customer_id=customer_id_use, max_rating=4)

Empty DataFrame
Columns: [movie_title, rating]
Index: []
average rating 11376    4.4
Name: avg_rating, dtype: float64


In [29]:
get_users_loved_hated_movies(customer_id=customer_id_use, minmax_rating=4)

User 84100 loved these movies
Something's Gotta Give
X2: X-Men United
Bridget Jones's Diary
House of Sand and Fog
Kill Bill: Vol. 2

and disliked these movies


In [30]:

data_rating

,movie_id,customer_id,rating
0,1,1488844,3.0
1,1,822109,5.0
2,1,885013,4.0
3,1,30878,4.0
4,1,823519,3.0
...,...,...,...
2499995,468,1404021,5.0
2499996,468,544108,3.0
2499997,468,1071584,4.0
2499998,468,256747,3.0


## Recommendations with collaborative filtering

In [31]:
def build_training_set(df):
    reader = Reader()
    svd = SVD()
    data_new = Dataset.load_from_df(df[['customer_id', 'movie_id', 'rating']], reader)
    trainset = data_new.build_full_trainset()
    svd.fit(trainset)
    return trainset, svd

def get_summary(type="movie_id"):
    #>IF MOVIE:  movie_id, movie rating count, movie rating mean
    #>IF CUSTOMER: customer_id, custumer rating count, customer rating mean
    df_count_mean_summary = data_rating.groupby(type)['rating'].agg(['count', 'mean'])
    df_count_mean_summary.index = df_count_mean_summary.index.map(int)
    #>IF MOVIE: returns 1799.0 as a benchmark number
    #IF CUSTOMER: returns 52.0 as a benchmark number
    benchmark = round(df_count_mean_summary['count'].quantile(0.7),0)
    # drop all rows below benchmark
    df_drop_list = df_count_mean_summary[df_count_mean_summary['count'] < benchmark]
    # return all indexes to drop
    return df_drop_list

def get_customer_recommendations(customer_id, predictor):
    #> returns movie_id, movie_year, movie_title
    chosen_customer_pred = data_movies.copy()
    # fails if movie_id is the index so we have to reset the index back to normal (0-N)
    chosen_customer_pred = chosen_customer_pred.reset_index()
    # makes sure that we only pick movies that are not in the movie dropped list
    chosen_customer_pred = chosen_customer_pred[~chosen_customer_pred['movie_id'].isin(df_movie_drop_list)]
    # make prediction for customer with id = <customer_id> and put it into 'estimated_score'
    chosen_customer_pred['estimated_score'] = chosen_customer_pred['movie_id'].apply(lambda x: predictor.predict(customer_id, x).est)
    # sort by 'estimated score'
    chosen_customer_pred = chosen_customer_pred.sort_values('estimated_score', ascending=False).set_index('movie_id')
    return chosen_customer_pred

def display_rated_content(customer_id=customer_id_use, number_to_show=20):
    print("Movies/TV Shows rated by customer", customer_id)
    df = all_customers_id_plus_movie_title_rows(customer_id=customer_id)
    df = df[['movie_title', 'rating']].sort_values('rating', ascending=False)
    print(df[0:number_to_show].set_index('movie_title'))

def display_customers_recommendations(customer_id=customer_id_use, df=[], number_to_show=20):
    print("Movies/TV Shows recommended to customer")
    print(df[['movie_title', 'estimated_score']][0:number_to_show].set_index('movie_title'))

def display_history_plus_recommended(customer_id=customer_id_use, number_to_show=25):
    display_rated_content(customer_id=customer_id, number_to_show=number_to_show)
    chosen_customer_pred = get_customer_recommendations(customer_id=customer_id, predictor=svd)
    display_customers_recommendations(customer_id=customer_id, df=chosen_customer_pred, number_to_show=number_to_show)

def get_evaluation_and_svd(use_pickle=True):
    evaluationData_file_name = "evaluationData_build_training_set"
    svd_file_name = "svd_build_training_set"
    if(use_pickle == True):
        evaluationData = load_pickle(evaluationData_file_name)
        svd = load_pickle(svd_file_name)
        return evaluationData, svd
    else:
        evaluationData, svd = build_training_set(data_rating)
        save_to_pickle(evaluationData_file_name, evaluationData)
        save_to_pickle(svd_file_name, svd)
        return evaluationData, svd

In [32]:
evaluationData, svd = get_evaluation_and_svd(use_pickle=use_pickle_file)

In [33]:
## creating a list of the movies that we don't wanna include
df_movie_drop_list = get_summary(type="movie_id")
#df_customer_drop_list = get_summary(type="customer_id")

In [34]:
display_history_plus_recommended(customer_id=79724, number_to_show=10)

Movies/TV Shows rated by customer 79724
                            rating
movie_title                       
Dogma                          5.0
What the #$*! Do We Know!?     4.0
Reservoir Dogs                 4.0
Ed Wood                        3.0
Pay It Forward                 3.0
Igby Goes Down                 3.0
Elephant                       2.0
Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
Aqua Teen Hunger Force: Vol. 1                             5.000000
Lord of the Rings: The Return of the King: Exte...         4.490710
Rudolph the Red-Nosed Reindeer                             4.346710
ABC Primetime: Mel Gibson's The Passion of the ...         4.163294
The Weather Underground                                    4.129848
Boycott                                                    4.119695
Zatoichi's Conspiracy                                      4.

In [35]:
display_history_plus_recommended(customer_id=customer_id_use, number_to_show=10)

Movies/TV Shows rated by customer 84100
                        rating
movie_title                   
Something's Gotta Give     5.0
House of Sand and Fog      5.0
X2: X-Men United           4.0
Bridget Jones's Diary      4.0
Kill Bill: Vol. 2          4.0
Movies/TV Shows recommended to customer
                                                    estimated_score
movie_title                                                        
Lord of the Rings: The Return of the King: Exte...         4.657058
Spitfire Grill                                             4.183466
Zatoichi's Conspiracy                                      4.096977
Aqua Teen Hunger Force: Vol. 1                             4.050542
ABC Primetime: Mel Gibson's The Passion of the ...         4.047677
Inspector Morse 31: Death Is Now My Neighbour              4.040900
Dinosaur Planet                                            3.982314
Lilo and Stitch                                            3.953918
The Rise and Fall of EC

In [36]:
#TODO:
#for the new user problem
## find highest rated movie / tv shows that have been watched the most -> how likely is new user to click on it ?

## SOMETHING

In [37]:
from surprise import dataset
from surprise import KNNBaseline
from surprise.model_selection import train_test_split 
from surprise.model_selection import LeaveOneOut
from surprise import accuracy

In [38]:
def build_training_set_same_as_above(df):
    reader = Reader()
    svd = SVD()
    data_new = Dataset.load_from_df(df[['customer_id', 'movie_id', 'rating']], reader)
    trainset = data_new.build_full_trainset()
    svd.fit(trainset)
    return trainset, svd

def print_evaluation_accuracy(prediction):
    print("\nEvaluating accuracy of model...")
    print("RMSE: ", accuracy.rmse(prediction, verbose=False))
    print("MAE: ", accuracy.mae(prediction, verbose=False))

In [39]:
reader = Reader(line_format='user item rating', rating_scale=(1, 5))
class convert_to_raw_ratings(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in zip(df['customer_id'], df['movie_id'], df['rating'])]
        self.reader=reader

raw_ratings = convert_to_raw_ratings(data_rating, reader)
print("\nBuilding recommendation model...")
trainSet, testSet = train_test_split(raw_ratings, test_size=.25, random_state=1)


Building recommendation model...


In [40]:
algo = SVD(random_state=10)
algo.fit(trainSet)

print("\nComputing recommendations...")
predictions = algo.test(testSet)


Computing recommendations...


In [41]:
cross_validate(algo,  raw_ratings, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9888  0.9902  0.9898  0.9902  0.9916  0.9901  0.0009  
MAE (testset)     0.7784  0.7787  0.7792  0.7786  0.7789  0.7787  0.0003  
Fit time          93.36   95.67   94.75   91.04   91.13   93.19   1.87    
Test time         4.59    3.12    4.12    4.15    4.15    4.02    0.49    


{'test_rmse': array([0.9887969 , 0.99020735, 0.98982276, 0.99020268, 0.99156106]),
 'test_mae': array([0.77835025, 0.77868123, 0.77916855, 0.77860402, 0.77885895]),
 'fit_time': (93.35542702674866,
  95.67373394966125,
  94.74725008010864,
  91.03855204582214,
  91.12549901008606),
 'test_time': (4.58624267578125,
  3.118312358856201,
  4.116436004638672,
  4.154078006744385,
  4.148710250854492)}

In [42]:
print_evaluation_accuracy(predictions)


Evaluating accuracy of model...
RMSE:  0.9912543460061926
MAE:  0.7797843476506224


In [43]:
print_evaluation_accuracy(svd.test(testSet))


Evaluating accuracy of model...
RMSE:  1.0386634945860738
MAE:  0.8522688604750867


## other

In [44]:
reader = Reader(line_format='user item rating', rating_scale=(1, 5))
dataset = Dataset.load_from_df(data_rating[['customer_id', 'movie_id', 'rating']], reader)
trainset = dataset.build_full_trainset()

In [45]:
tmp_data_movies = data_movies[['movie_id', 'movie_title']]
tmp_data_movies = tmp_data_movies.set_index('movie_id').T
tmp_data_movies = tmp_data_movies.to_dict('list')
tmp_data_movies = {k: str(v[0]) for k,v in tmp_data_movies.items()}

In [46]:
similarity_matrix = KNNBasic(sim_options={'name': 'cosine', 'user_based': False}).fit(trainset).compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [47]:
# generate_random_user_ids(1000, 1500)

In [48]:
# for i in generate_random_user_ids(1000, 1500):
#     if (len(all_custumers_id_rows(i)) > 3):
#         print(i)

In [49]:
test_subject = customer_id_use#16272
k = 10

In [50]:
test_subject_iid = trainset.to_inner_uid(test_subject)

test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbours = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [51]:
candidates = defaultdict(float)

for itemID, rating in k_neighbours:
    try:
        similarities = similarity_matrix[itemID]
        for innerID, score in enumerate(similarities):
            candidates[innerID] += score * (rating / 5.0)
    except:
        continue

In [52]:
def getMovieName(movieID):
  if int(movieID) in tmp_data_movies:
    return tmp_data_movies[int(movieID)]
  else:
      return ""

In [53]:
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
    watched[itemID] = 1

recommendations = []
position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        recommendations.append(getMovieName(trainset.to_raw_iid(itemID)))
        position += 1
        # only want top n which in our case in 10
        if(position > 10): break

for rec in recommendations:
    print("Movie: ", rec)


Movie:  Lord of the Rings: The Return of the King: Extended Edition: Bonus Material
Movie:  Mississippi Burning
Movie:  North by Northwest
Movie:  Silkwood
Movie:  Rabbit-Proof Fence
Movie:  Woman of the Year
Movie:  Elfen Lied
Movie:  The Chorus
Movie:  Sex and the City: Season 4
Movie:  Parenthood
Movie:  Charlotte's Web


In [54]:
# #TODO: keep or throw away since it's never used
# sim_options = {'name': 'pearson_baseline', 'user_based': False}
# simsAlgo = KNNBaseline(sim_options=sim_options)
# simsAlgo.fit(evaluationData)